# for zip file

In [ ]:
import zipfile
import os

# Set the path to the uploaded ZIP file
zip_path = '/content/try-1.zip'  # Update this if your file has a different name
extract_path = '/content/dataset'  # Destination folder

# Create the destination directory if it doesn't exist
os.makedirs(extract_path, exist_ok=True)

# Unzip the dataset
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Dataset extracted to {extract_path}")

Dataset extracted to /content/dataset


# this also for all frames

In [ ]:
import cv2
import os

# === CONFIGURATION ===
video_path = "/content/VC Video.mp4"  # <-- Replace with your actual video path
output_dir = "/content/dataset/images/train"
total_frames_expected = 2391  # frames 0 to 2390
image_format = "jpg"

# === CREATE OUTPUT FOLDER ===
os.makedirs(output_dir, exist_ok=True)

# === READ VIDEO ===
cap = cv2.VideoCapture(video_path)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

if frame_count < total_frames_expected:
    print(f"Warning: video only has {frame_count} frames, expected {total_frames_expected}")
elif frame_count > total_frames_expected:
    print(f"Note: video has {frame_count} frames, but CVAT used only 2391 (0–2390)")

print("Extracting frames...")

frame_idx = 0
while frame_idx < total_frames_expected:
    success, frame = cap.read()
    if not success:
        print(f"Error reading frame at index {frame_idx}")
        break
    filename = f"frame_{frame_idx:06d}.{image_format}"  # ← MATCHING the annotation file names
    cv2.imwrite(os.path.join(output_dir, filename), frame)
    frame_idx += 1

cap.release()
print(f"Done: {frame_idx} frames extracted to {output_dir}")


Extracting frames...
Done: 2391 frames extracted to /content/dataset/images/train


# this for only labelled frames

In [ ]:
import cv2
import os

# === CONFIGURATION ===
video_path = "/content/VC Video.mp4"  # Path to your video
labels_dir = "/content/dataset/labels/train"  # Label files directory
output_dir = "/content/dataset/images/train"  # Where to save extracted frames
image_format = "jpg"

# === COLLECT FRAME INDICES FROM LABEL FILES ===
label_files = [f for f in os.listdir(labels_dir) if f.endswith('.txt')]
frame_indices = sorted([
    int(f.split('_')[1].split('.')[0]) for f in label_files
])  # Extract frame numbers like 000117 → 117

print(f"{len(frame_indices)} labeled frames found (from {min(frame_indices)} to {max(frame_indices)}).")

# === CREATE OUTPUT FOLDER ===
os.makedirs(output_dir, exist_ok=True)

# === READ VIDEO ===
cap = cv2.VideoCapture(video_path)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print("Extracting only labeled frames...")

frame_pos = 0
extracted = 0
target_set = set(frame_indices)

while frame_pos <= max(frame_indices):
    success, frame = cap.read()
    if not success:
        print(f"Error reading frame at index {frame_pos}")
        break

    if frame_pos in target_set:
        filename = f"frame_{frame_pos:06d}.{image_format}"
        cv2.imwrite(os.path.join(output_dir, filename), frame)
        extracted += 1

    frame_pos += 1

cap.release()
print(f"Done: {extracted} labeled frames extracted to {output_dir}")


# this also only for labelled ones

In [ ]:
import cv2
import os
import yaml

# === CONFIGURATION ===
image_dir = "/content/dataset/images/train"
label_dir = "/content/dataset/labels/train"
data_yaml_path = "/content/dataset/data.yaml"
output_video_path = "/content/output_annotated_video.mp4"
image_format = "jpg"
fps = 30  # Set based on your original video

# === LOAD CLASS NAMES FROM YAML ===
with open(data_yaml_path, 'r') as f:
    data = yaml.safe_load(f)
class_names = data['names']

# === COLLECT IMAGE FILES ===
image_files = sorted([
    f for f in os.listdir(image_dir) if f.endswith(f".{image_format}")
])

if not image_files:
    raise FileNotFoundError("No image files found in image_dir.")

# === PREPARE VIDEO WRITER ===
first_frame = cv2.imread(os.path.join(image_dir, image_files[0]))
frame_height, frame_width = first_frame.shape[:2]
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# === FUNCTION TO DRAW BBOXES ===
def draw_annotations(image, label_path):
    h, w = image.shape[:2]
    if not os.path.exists(label_path):
        return image  # No annotation
    with open(label_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) != 5:
                continue
            cls_id, x_c, y_c, box_w, box_h = map(float, parts)
            x1 = int((x_c - box_w / 2) * w)
            y1 = int((y_c - box_h / 2) * h)
            x2 = int((x_c + box_w / 2) * w)
            y2 = int((y_c + box_h / 2) * h)
            cls_name = class_names[int(cls_id)]
            color = (0, 255, 0)
            cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
            cv2.putText(image, cls_name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    return image

# === PROCESS AND SAVE VIDEO ===
print("Rendering annotated video...")

for image_file in image_files:
    frame_path = os.path.join(image_dir, image_file)
    frame = cv2.imread(frame_path)
    if frame is None:
        print(f"Warning: Could not read {frame_path}")
        continue

    frame_id = os.path.splitext(image_file)[0]  # e.g., "frame_000117"
    label_path = os.path.join(label_dir, f"{frame_id}.txt")
    annotated = draw_annotations(frame, label_path)
    out.write(annotated)

out.release()
print(f"✅ Annotated video saved at: {output_video_path}")


Rendering annotated video...
✅ Annotated video saved at: /content/output_annotated_video.mp4


# this for all frames


In [ ]:
import cv2
import os
import yaml

# === CONFIGURATION ===
image_dir = "/content/dataset/images/train"
label_dir = "/content/dataset/labels/train"
data_yaml_path = "/content/dataset/data.yaml"
output_video_path = "/content/output_annotated_video.mp4"
image_format = "jpg"
fps = 30  # Adjust to match your original video
total_frames = 2391  # 0 to 2390

# === LOAD CLASS NAMES FROM YAML ===
with open(data_yaml_path, 'r') as f:
    data = yaml.safe_load(f)
class_names = data['names']

# === PREPARE VIDEO WRITER ===
first_frame_path = os.path.join(image_dir, f"frame_{0:06d}.{image_format}")
first_frame = cv2.imread(first_frame_path)
if first_frame is None:
    raise FileNotFoundError(f"First frame not found at {first_frame_path}")
frame_height, frame_width = first_frame.shape[:2]
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# === FUNCTION TO DRAW BBOXES ===
def draw_annotations(image, label_path):
    h, w = image.shape[:2]
    if not os.path.exists(label_path):
        return image  # No annotation
    with open(label_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) != 5:
                continue  # Skip invalid lines
            cls_id, x_c, y_c, box_w, box_h = map(float, parts)
            x1 = int((x_c - box_w / 2) * w)
            y1 = int((y_c - box_h / 2) * h)
            x2 = int((x_c + box_w / 2) * w)
            y2 = int((y_c + box_h / 2) * h)
            cls_name = class_names[int(cls_id)]
            color = (0, 255, 0)
            cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
            cv2.putText(image, cls_name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    return image

# === PROCESS AND SAVE VIDEO ===
print("Rendering annotated video...")
for i in range(total_frames):
    img_path = os.path.join(image_dir, f"frame_{i:06d}.{image_format}")
    label_path = os.path.join(label_dir, f"frame_{i:06d}.txt")
    frame = cv2.imread(img_path)
    if frame is None:
        print(f"Missing frame: {img_path}")
        continue
    annotated = draw_annotations(frame, label_path)
    out.write(annotated)

out.release()
print(f"✅ Annotated video saved at: {output_video_path}")


# This for downloading that file

In [ ]:
from google.colab import files
import shutil

# Zip the dataset folder (recursively)
shutil.make_archive('/content/dataset_new', 'zip', '/content/dataset')

# Download the zipped dataset
files.download('/content/dataset_new.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>